In [1]:
import requests
from scrapy.selector import Selector
import re

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0'

In [2]:
# список аниме

In [3]:
response = requests.get("https://jut.su/anime/",
                       headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0',
    })

In [4]:
content = response.content.decode('cp1251').encode('utf8')

In [5]:
x = Selector(text=content).xpath('//div[contains(@class, "all_anime_global")]')

In [6]:
refs = x.xpath('./a/@href').getall()
names = x.xpath('.//div[contains(@class, "aaname")]/text()').getall()
x_series = x.xpath('.//div[contains(@class, "aailines")]')

def get_series_cnt(content:Selector)->int:
    text = "".join(content.xpath("./text()").getall())
    series_cnt = re.search(r"(\d+)\sсери", text).group(1)
    return int(series_cnt) if series_cnt else 0

series_cnts = [get_series_cnt(item) for item in x_series]

In [7]:
anime_dict = {}
for ref, name, series_cnt in zip(refs, names, series_cnts):
    if ref not in anime_dict:
        anime_dict[ref] = {"ref":ref, "name":name, "series_cnt":series_cnt}

In [8]:
selected = ['/full-metal-alchemist/']
selected_dict = {key:item for key, item in anime_dict.items() if key in selected}

In [9]:
# for key in selected:
#     response = requests.get(f"https://jut.su{key}",
#                        headers = {
#         'User-Agent': user_agent
#     })
#     content = response.content.decode('cp1251').encode('utf8')
#     series = Selector(text=response.content).xpath('//a[contains(@class, "short-btn")]//@href')
#     diff = len(series) - anime_dict[key]["n_series"]
#     if  diff!= 0:
#         name = anime_dict[key]["name"]
#         print(f"{name} {diff} новых серий!")
#     anime_dict[key]["n_series"] = len(series)
# # TODO descrpition
# # x.xpath('.//div[contains(@class, "tooltip_of_the_anime")]/@content').get()

In [10]:
# class AnimeObserver():
#     def __init__(self, selected):
#         self.selected = selected
        

In [11]:
selected_item = selected_dict['/full-metal-alchemist/']


In [15]:
from telegram.ext.updater import Updater
from telegram.update import Update
from telegram.ext.callbackcontext import CallbackContext
from telegram.ext.commandhandler import CommandHandler
from telegram.ext.messagehandler import MessageHandler
from telegram.ext.filters import Filters

updater = Updater("token", use_context=True)


def start(update: Update, context: CallbackContext):
    update.message.reply_text(
        "Hello sir, Welcome to the Bot.Please write\
        /help to see the commands available.")

def help(update: Update, context: CallbackContext):
    update.message.reply_text("""Available Commands :-
    /add - Add selected series
    /remove - Remove series from selected list
    """)


def add(update: Update, context: CallbackContext):
    update.message.reply_text(
        f"{selected_item['name']}: {selected_item['series_cnt']}")

def remove(update: Update, context: CallbackContext):
    update.message.reply_text(
        "Remove items")





def unknown(update: Update, context: CallbackContext):
    update.message.reply_text(
        "Sorry '%s' is not a valid command" % update.message.text)


def unknown_text(update: Update, context: CallbackContext):
    update.message.reply_text(
        "Sorry I can't recognize you , you said '%s'" % update.message.text)


updater.dispatcher.add_handler(CommandHandler('start', start))
updater.dispatcher.add_handler(CommandHandler('add', add))
updater.dispatcher.add_handler(CommandHandler('remove', remove))
updater.dispatcher.add_handler(CommandHandler('help', help))
updater.dispatcher.add_handler(MessageHandler(Filters.text, unknown))
updater.dispatcher.add_handler(MessageHandler(
    Filters.command, unknown)) # Filters out unknown commands

# Filters out unknown messages.
updater.dispatcher.add_handler(MessageHandler(Filters.text, unknown_text))

updater.start_polling()


Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot5653184865:AAGwa1fcckpNAgxDJvLXdeXxRccVrgJElfA/getUpdates (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x00000178F9262F40>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "C:\Users\migareev\Anaconda3\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\connection.py", line 140, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\migareev\Anaconda3\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\util\connection.py", line 83, in create_connection
    raise err
  File "C:\Users\migareev\Anaconda3\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
socket.timeout: timed out



Error while getting Updates: urllib3 HTTPError HTTPSConnectionPool(host='api.telegram.org', port=443): Max retries exceeded with url: /bot5653184865:AAGwa1fcckpNAgxDJvLXdeXxRccVrgJElfA/getUpdates (Caused by ConnectTimeoutError(<telegram.vendor.ptb_urllib3.urllib3.connection.VerifiedHTTPSConnection object at 0x00000178F8D6D8E0>, 'Connection to api.telegram.org timed out. (connect timeout=5.0)'))
No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "C:\Users\migareev\Anaconda3\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\connection.py", line 140, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\migareev\Anaconda3\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\util\connection.py", line 83, in create_connection
    raise err
  File "C:\Users\migareev\Anaconda3\lib\site-packages\telegram\vendor\ptb_urllib3\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
socket.timeout: timed out



In [14]:
updater.stop()